# Böhringer - Accumulations

## Info
* Achtung: Spalte "Total" in RnD ist Total über das gesammte PDF!

In [1]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

In [2]:
# Read pdf into DataFrame
df = tabula.read_pdf("switzerland_scienceindustries_de_fr_en_24jun2020.pdf", pages='all', lattice=True)

In [9]:
df_export = df.copy()

#Shift all where column[1] not empty
column0 = df_export[df_export.columns[0]]
df_export[column0.isna()] = df_export[column0.isna()].shift(-1, axis='columns')

#Drop Columns
df_export = df_export.drop(df.columns[9:14],axis=1)
df_export = df_export.drop(df.columns[7],axis=1)

#Rename
df_export.columns = fix_columns_accumulations[0: 8]

#fill na
df_export = df_export.fillna("")

# Select
amount = df_export[df_export.type.str.contains('Gesamtbetrag entfallend')]
count = df_export[df_export.type.str.contains('Anzahl Empfänger in')]
rnd = df_export[df_export.type.str.contains('Geldwerte Leistungen')]

df_export = pd.concat([amount, count, rnd])

#Numberize and sum
df_export = cleanup_number(df_export)
df_export = replace_comma_to_dot(df_export)
df_export = amounts_to_number(df_export)

#Rename first column
df_export = df_export.reset_index(drop=True)
df_export.iloc[0,0] = "hcp_amount"
df_export.iloc[1,0] = "hco_amount"
df_export.iloc[2,0] = "hcp_count"
df_export.iloc[3,0] = "hco_count"
df_export.iloc[4,0] = "rnd"

df_export.loc[df_export.type == 'rnd', "total"] = df_export["donations_grants"]
df_export.loc[df_export.type == 'rnd', "donations_grants"] = np.NaN
df_export.loc[df_export.type == 'rnd', "sponsorship"] = np.NaN

export_acumulations(df_export, 'boehringer')


saved


In [ ]:
#write_to_excel(df_export, 'tmp.xlsx', True)